In [1]:
from fastai.vision.all import *
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.tracker import SaveModelCallback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib2 import Path
import numpy as np
import random
from torch.nn import MSELoss

In [2]:
%%time

df = pd.read_csv('data/examples.csv')
df.shape

CPU times: user 22.8 s, sys: 2.37 s, total: 25.1 s
Wall time: 25.1 s


(14276908, 9)

In [3]:
%%time

uniq_fns = np.unique(df.target_fn.values)
fn2feature = {}
for fn in uniq_fns:
    ary = pd.read_pickle(f'data/examples/{fn}.pkl')
    fn2feature[fn] = ary

pd.to_pickle(fn2feature, 'data/fn2feature.pkl')

CPU times: user 24min 7s, sys: 5min 22s, total: 29min 30s
Wall time: 1h 14min 34s


In [4]:
%%time
fn2features = pd.read_pickle('data/fn2feature.pkl')

CPU times: user 17 s, sys: 6.76 s, total: 23.8 s
Wall time: 23.8 s


In [5]:
%%time
fn2feature_length = {}

for fn, feature in fn2features.items():
    fn2feature_length[fn] = feature.shape[0]

CPU times: user 2.87 s, sys: 0 ns, total: 2.87 s
Wall time: 2.87 s


In [6]:
%%time

df['source_length'] = df.source_fn.apply(lambda fn: fn2feature_length[fn])
df['target_length'] = df.target_fn.apply(lambda fn: fn2feature_length[fn])

CPU times: user 15.9 s, sys: 616 ms, total: 16.6 s
Wall time: 16.6 s


In [7]:
df.to_csv('data/examples_with_length.csv', index=False)

In [8]:
((df.source_length < 70) & (df.target_length < 70)).mean()

0.95829040853944

In [9]:
df.source_word.nunique()

64763

In [10]:
df[((df.source_length < 70) & (df.target_length < 70))].source_word.nunique()

55687

In [11]:
df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 25))].drop_duplicates(subset='source_fn').sort_values('source_length', ascending=True).head(20)

,source_word,target_word,source_fn,target_fn,set_name,speaker_id,book_id,distance_from_target,audio_fpath,source_length,target_length
9556703,THREW,THE,1851781743f5452bb514b6b68dca6b58,0a387c5255c14cf581ca48ae2bb00c56,train-clean-360,7752,110522,2,data/LibriSpeech/train-clean-360/7752/110522/7752-110522-0028.flac,26,12
3573581,LAY,LATTER,f98b258034df4ed29beaccdbb9f5e1cb,556a2c62f13c493abb4cf00d1284e885,train-clean-360,3446,176270,2,data/LibriSpeech/train-clean-360/3446/176270/3446-176270-0039.flac,26,40
4780572,HER,SOON,27fc84bf397b4ff49aac1a38589b0a47,f1d0a4545f0b4524b2a7298d9501f566,train-clean-360,8776,270778,2,data/LibriSpeech/train-clean-360/8776/270778/8776-270778-0009.flac,26,29
6463180,OVER,THE,58622c8a52e44737a8b64fa6a350d5c5,a82c471a0a724c88bc22327f7061f669,train-clean-360,1066,103481,1,data/LibriSpeech/train-clean-360/1066/103481/1066-103481-0024.flac,26,9
10875612,AND,FIGURES,90fdf494668b443ca252b1813b0f811f,551e03c5a3ee4568b443a3f3c39a8a96,train-clean-360,1046,133225,1,data/LibriSpeech/train-clean-360/1046/133225/1046-133225-0060.flac,26,62
13086737,GOOD,BE,4b8198cf449244d49e89538d5b201940,61895b5ddcd94ec4b16df466c0a5163e,train-clean-100,5688,15787,2,data/LibriSpeech/train-clean-100/5688/15787/5688-15787-0034.flac,26,12
13086750,TOO,MUCH,199619243cac4c4dbc83ff65856597b6,31f01fca890b4e4085c6ac58644e6c76,train-clean-100,5688,15787,1,data/LibriSpeech/train-clean-100/5688/15787/5688-15787-0035.flac,26,27
706289,QUITE,KEEP,31c033526c8144e5ac29d04aa7bac46f,21cf3c5ea3724895ad3a846589e445a9,train-clean-360,8008,271817,2,data/LibriSpeech/train-clean-360/8008/271817/8008-271817-0020.flac,26,26
706281,KEEP,CAPTAIN,21cf3c5ea3724895ad3a846589e445a9,0ad2481f02a642d2905851686b9b9e84,train-clean-360,8008,271817,2,data/LibriSpeech/train-clean-360/8008/271817/8008-271817-0020.flac,26,43
10875506,ONE,EIGHTY,7db650a6c87942879ff22412f9dfc19e,094403ecb3754a61851e6ff14e711816,train-clean-360,1046,133225,1,data/LibriSpeech/train-clean-360/1046/133225/1046-133225-0058.flac,26,30


In [12]:
from IPython.lib.display import Audio

In [14]:
Audio(filename='data/examples_audio/1e983242c6534928a06abdf9c12159a1.wav')

In [15]:
df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 20))].source_word.nunique()

55463